# A study of traffic crashes in the city of Tempe, Arizona USA
***

## Motivation
<span style="color:red">

- What is your dataset?
- Why did you choose this/these particular dataset(s)?
- What was your goal for the end user's experience?
</span>


## Basic stats
<span style="color:red">
Let's understand the dataset better

- Write about your choices in data cleaning and preprocessing
- Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.
</span>

In [ ]:
import numpy as np
import urllib.request
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import calplot
import seaborn as sns
from datetime import datetime, timedelta
import math
from math import radians, cos, sin, asin, sqrt

import folium

from urllib.request import urlopen
import json
import plotly.express as px

# Import libraries to make heatmap
from folium import plugins
from folium.plugins import HeatMap


In [ ]:
# Read data 
data = pd.read_csv("1.08_Crash_Data_Report_(detail).csv")

# subtract 7 hours
data['DateTime'] = pd.to_datetime(data['DateTime']) - timedelta(hours=7)

# Filter the DataFrame to exclude entries from the year 2024
df = data[data['DateTime'].dt.year != 2024] 

# Filter the DataFrame to exclude entries from the year 2023 incomplete
df = data[data['DateTime'].dt.year != 2023] 


## Data Analysis
<span style="color:red">

- Describe your data analysis and explain what you've learned about the dataset.
- If relevant, talk about your machine-learning.
</span>


## Genre
<span style="color:red">

- Which genre of data story did you use?
- Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
- Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?</span>



## Visualizations

<span style="color:red">

- Explain the visualizations you've chosen.
- Why are they right for the story you want to tell?
</span>


## Discussion
<span style="color:red">
Think critically about your creation

- What went well?,
- What is still missing? What could be improved?, Why? </span>



## Contributions
<span style="color:red">
 Who did what?

- You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).
- It is not OK simply to write "All group members contributed equally". </span>


Section/Name  | Lau           | Anna          | Siv 
------------- | ------------- | ------------- | -------------
| merge right       |
Content Cell  | Content Cell  | Content Cell  | Content Cell
Content Cell  | Content Cell  | Content Cell  | Content Cell


## References

<span style="color:red">

- Make sure that you use references when they're needed and follow academic standards.</span>